# Generate VIIRS points

**Name:** Asaf Vanunu  
**Date:** August 21, 2024  
**step:** 1

## Description
This script takes VIIRS fire product and create VIIRS shapefile points while taking into account duplicated fire events resulting from the VIIRS bowtie effect.


* Load libraries

In [2]:
import os
import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapely.geometry
import GOES_VIIRS_tools
from rasterio.plot import show

* First we will set a path for VIIRS data

In [3]:
VIIRS_fire_path = "F:\\VIIRS_data\\VIIRS_fire"
VIIRS_files = [os.path.join(VIIRS_fire_path, f) for f in os.listdir(VIIRS_fire_path) if f.endswith('.nc')] ## get all the VIIRS files

In [36]:
def conf_to_str(conf):
    """This function converts the confidence value to a string

    Args:
        conf (int): The confidence value

    Returns:
        str: the string representation of the confidence value
    """
    if conf == 7:
        return "l"
    elif conf == 8:
        return "n"
    elif conf == 9:
        return "h"

* We will need to create a function that generates VIIRS points just from the fire file

In [73]:
def generate_VIIRS_points(VIIRS_fire_file_path):
    """This function generates a geodataframe of the VIIRS fire data from VIIRS NetCDF file.

    Args:
        VIIRS_fire_file_path (str): Full path to the VIIRS NetCDF file.
    """
    confidences_names = ["l", "n", "h"]
    VIIRS = xr.open_dataset(VIIRS_fire_file_path) ## open the VIIRS file
    number_of_fire_pixels = len(VIIRS["FP_latitude"].values) ## get the number of fire pixels
    VIIRS_file_name = os.path.basename(VIIRS_fire_file_path) ## get the file name
    VIIRS_file_name_list = np.repeat(VIIRS_file_name, number_of_fire_pixels) ## repeat the file name
    VIIRS_unique_date = VIIRS_file_name.split(".")[1] ## get the unique date, example: 'A2022152'
    VIIRS_unique_date = np.repeat(VIIRS_unique_date, number_of_fire_pixels) ## repeat the unique date
    VIIRS_time = GOES_VIIRS_tools.generate_VIIRS_time_from_image(VIIRS_fire_path=VIIRS_fire_file_path,
                                                                 type_of_date="time")
    VIIRS_time_list = np.repeat(VIIRS_time, number_of_fire_pixels) ## get the unique time. example: '09:06'
    VIIRS_time_split = VIIRS_time.split(":") ## split the time
    VIIRS_unique_time = f"{VIIRS_time_split[0]}{VIIRS_time_split[1]}" ## get the unique time. example: "0906"
    VIIRS_unique_time = np.repeat(VIIRS_unique_time, number_of_fire_pixels) ## repeat the unique time
    VIIRS_date = GOES_VIIRS_tools.generate_VIIRS_time_from_image(VIIRS_fire_path=VIIRS_fire_file_path,
                                                                 type_of_date="date") ## get the unique date. format: '2021-06-01'
    VIIRS_date = np.repeat(VIIRS_date, number_of_fire_pixels) ## repeat the unique date
    VIIRS_date_time = GOES_VIIRS_tools.generate_VIIRS_time_from_image(VIIRS_fire_path=VIIRS_fire_file_path,
                                                                      type_of_date="date_time") ## get the unique date time format: '2021-06-01 09:06'
    VIIRS_date_time = np.repeat(VIIRS_date_time, number_of_fire_pixels) ## repeat the unique date time
    VIIRS_day_night = VIIRS.attrs['DayNightFlag'] ## get the day night flag
    VIIRS_day_night = np.repeat(VIIRS_day_night, number_of_fire_pixels) ## repeat the day night flag
    
    ####### Now we will set the array variables ########
    
    VIIRS_lat = VIIRS["FP_latitude"].values ## get the latitude values
    VIIRS_lon = VIIRS["FP_longitude"].values ## get the longitude values
    VIIRS_fire_pixels = VIIRS["FP_confidence"].values ## get the fire pixels
    VIIRS_fire_pixels_name = list(map(conf_to_str, VIIRS_fire_pixels)) ## get the fire pixels names 
    VIIRS_scan_line = VIIRS["FP_line"].values ## get the scan line
    VIIRS_azimuth_angle = VIIRS["FP_ViewZenAng"].values ## get the azimuth angle
    VIIRS_grid_sample = VIIRS["FP_sample"].values ## get the grid sample
    
    ######## Now we will create the geodataframe ########
    VIIRS_dict = {"latitude": VIIRS_lat, "longitude": VIIRS_lon,
                  "Fire_file": VIIRS_file_name_list,
                  "Unique_date": VIIRS_unique_date,
                  "Unique_time": VIIRS_unique_time,
                  "DATE": VIIRS_date,
                  "TIME": VIIRS_time_list,
                  "DATE_TIME": VIIRS_date_time,
                  "fire_pixel_confidence": VIIRS_fire_pixels,
                  "fire_pixel_confidence_str": VIIRS_fire_pixels_name,
                  "scan_line": VIIRS_scan_line,
                  "grid_sample": VIIRS_grid_sample,
                  "View_Zenith_Angle": VIIRS_azimuth_angle,
                  "night/day": VIIRS_day_night}
    VIIRS_df = pd.DataFrame(VIIRS_dict) ## create a dataframe
    
    gdf_VIIRS = gpd.GeoDataFrame(
        VIIRS_df, geometry=gpd.points_from_xy(VIIRS_df.longitude, VIIRS_df.latitude), crs="EPSG:4326") ## create a geodataframe
    
    return gdf_VIIRS    

* Now we will create a function to remove duplicated fire events resulting from the VIIRS bowtie effect
* In addition it will remove low confidence fire events

In [75]:
def remove_duplicate_VIIRS_fires(VIIRS_gdf, buffer_size=300, scan_line_diff=3):
    """This function removes duplicated VIIRS fire pixels by creating a buffer around the fire pixel and removing all the fire pixels that are within the buffer.

    Args:
        VIIRS_gdf (GeoDataFrame): a VIIRS GeoDataFrame that contains the fire pixels
        buffer_size (int, optional): The size of the buffer. Defaults to 300.
        scan_line_diff (int, optional): The difference between scan lines .Defaults to 3.
    """
    fire_points_to_remove_id = [] ## create an empty list to store the fire points to remove
    NAD_1983_CRS = "ESRI:102008" ## For accurate distance calculations we need to use a projected coordinate system
    VIIRS_gdf = VIIRS_gdf.to_crs(NAD_1983_CRS) ## change the coordinate system of the GeoDataFrame
    VIIRS_gdf = VIIRS_gdf[VIIRS_gdf['fire_pixel_confidence'] != 7].reset_index(drop=True) ## remove the low confidence fire pixels
    con_zenith_view = VIIRS_gdf["View_Zenith_Angle"] >= 19 ## create a condition for the view zenith angle of the fire pixels where the view zenith angle is greater than 19
    gdf_filter = VIIRS_gdf[con_zenith_view].reset_index(drop=True) ## apply the condition
    con_zenith_view_no_dup = VIIRS_gdf["View_Zenith_Angle"] < 19 ## create a condition for the view zenith angle of the fire pixels where the view zenith angle is less than 19
    gdf_no_dup = VIIRS_gdf[con_zenith_view_no_dup].reset_index(drop=True) ## apply the condition
    gdf_filter["ID"] = gdf_filter.index ## create an ID column
    
    
    for i in range(len(gdf_filter)): ## loop through all the fire pixels
        fire_point = gdf_filter.iloc[[i]] ## get the fire pixel
        fire_point_id = fire_point["ID"].values[0] ## get the fire pixel ID
        if ~np.isin(fire_point_id, fire_points_to_remove_id): ## If the fire point is not used
            fire_point_scan_line = fire_point["scan_line"].iloc[0] ## get the scan line of the fire pixel
            buffer = fire_point.buffer(buffer_size) ## create a buffer around the fire pixel
            fire_point_buffer = gpd.GeoDataFrame(geometry=buffer, crs=NAD_1983_CRS) ## create a GeoDataFrame from the buffer
            buffer_intersection = gpd.overlay(gdf_filter, fire_point_buffer, how='intersection') ## get the intersection between the buffer and the fire pixels
            diff_con = abs(fire_point_scan_line - buffer_intersection["scan_line"].values.astype(int)) >= scan_line_diff ## create a condition to filter the fire pixels
            buffer_intersection_filter = buffer_intersection[diff_con] ## apply the condition
        
            if len(buffer_intersection_filter) > 0: ## if there are fire pixels in the buffer
                fire_points_to_remove_id.extend(buffer_intersection_filter["ID"].values) ## add the fire pixels in the buffer to the fire points to remove
        
    ## after the loop is done, remove the fire points that are in the fire points to remove list
    VIIRS_gdf_filtered = gdf_filter[~np.isin(gdf_filter["ID"], fire_points_to_remove_id)] ## filter the fire pixels
    VIIRS_gdf_filtered = VIIRS_gdf_filtered.drop(columns=["ID"]) ## drop the ID column
    if len(gdf_no_dup) > 0: ## if there are fire pixels with view zenith angle less than 19
        VIIRS_gdf_filtered = pd.concat([VIIRS_gdf_filtered, gdf_no_dup], ignore_index=True)
        VIIRS_gdf_filtered = VIIRS_gdf_filtered.to_crs("EPSG:4326") ## change the coordinate system back to WGS84
        return VIIRS_gdf_filtered
    else:
        VIIRS_gdf_filtered = VIIRS_gdf_filtered.to_crs("EPSG:4326") ## change the coordinate system back to WGS84
        return VIIRS_gdf_filtered ## return the filtered fire pixels
    

* Now we can generate VIIRS points

In [76]:
VIIRS_points = [] ## create an empty list to store the VIIRS points
for i in range(len(VIIRS_files)): ## loop through all the VIIRS files
    VIIRS_gdf = generate_VIIRS_points(VIIRS_files[i]) ## generate the VIIRS points
    VIIRS_gdf_filtered = remove_duplicate_VIIRS_fires(VIIRS_gdf=VIIRS_gdf, buffer_size=300, scan_line_diff=3) ## remove the duplicated VIIRS fires
    VIIRS_points.append(VIIRS_gdf_filtered) ## append the VIIRS points to the list
    print(f"VIIRS file {i+1} out of {len(VIIRS_files)} processed") ## print the progress

VIIRS file 1 out of 1171 processed
VIIRS file 2 out of 1171 processed
VIIRS file 3 out of 1171 processed
VIIRS file 4 out of 1171 processed
VIIRS file 5 out of 1171 processed
VIIRS file 6 out of 1171 processed
VIIRS file 7 out of 1171 processed
VIIRS file 8 out of 1171 processed
VIIRS file 9 out of 1171 processed
VIIRS file 10 out of 1171 processed
VIIRS file 11 out of 1171 processed
VIIRS file 12 out of 1171 processed
VIIRS file 13 out of 1171 processed
VIIRS file 14 out of 1171 processed
VIIRS file 15 out of 1171 processed
VIIRS file 16 out of 1171 processed
VIIRS file 17 out of 1171 processed
VIIRS file 18 out of 1171 processed
VIIRS file 19 out of 1171 processed
VIIRS file 20 out of 1171 processed
VIIRS file 21 out of 1171 processed
VIIRS file 22 out of 1171 processed
VIIRS file 23 out of 1171 processed
VIIRS file 24 out of 1171 processed
VIIRS file 25 out of 1171 processed
VIIRS file 26 out of 1171 processed
VIIRS file 27 out of 1171 processed
VIIRS file 28 out of 1171 processed
V

In [78]:
VIIRS_points_concat = pd.concat(VIIRS_points) ## concatenate the points
print(f"Total number of fire pixels: {len(VIIRS_points_concat)}") ## print the total number of fire pixels
VIIRS_points_concat[:5] ## show the first 5 rows

Total number of fire pixels: 227323


,latitude,longitude,Fire_file,Unique_date,Unique_time,DATE,TIME,DATE_TIME,fire_pixel_confidence,fire_pixel_confidence_str,scan_line,grid_sample,View_Zenith_Angle,night/day,geometry
0,62.578159,-105.049248,VNP14IMG.A2022152.0824.001.2022155015212.nc,A2022152,0824,2022-06-01,08:24,2022-06-01 08:24,8,n,2478,1358,51.085258,Both,POINT (-105.04925 62.57816)
1,58.585060,-116.972847,VNP14IMG.A2022152.0824.001.2022155015212.nc,A2022152,0824,2022-06-01,08:24,2022-06-01 08:24,8,n,3695,71,69.029762,Both,POINT (-116.97285 58.58506)
2,58.577908,-116.971733,VNP14IMG.A2022152.0824.001.2022155015212.nc,A2022152,0824,2022-06-01,08:24,2022-06-01 08:24,8,n,3696,71,69.029930,Both,POINT (-116.97173 58.57791)
3,57.405655,-111.052505,VNP14IMG.A2022152.0824.001.2022155015212.nc,A2022152,0824,2022-06-01,08:24,2022-06-01 08:24,8,n,4107,655,60.681606,Both,POINT (-111.05251 57.40565)
4,57.041893,-111.638710,VNP14IMG.A2022152.0824.001.2022155015212.nc,A2022152,0824,2022-06-01,08:24,2022-06-01 08:24,8,n,4212,584,61.619793,Both,POINT (-111.63871 57.04189)


* export the points to a shapefile

In [82]:
out_path = "F:\\VIIRS_data\\VIIRS_pnt_no_duplicates"
VIIRS_points_concat.to_file(os.path.join(out_path, "VIIRS_points_no_duplicates.shp")) ## save the VIIRS points to a shapefile

C:\Users\asaf_rs\AppData\Local\Temp\ipykernel_35876\801932299.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  VIIRS_points_concat.to_file(os.path.join(out_path, "VIIRS_points_no_duplicates.shp")) ## save the VIIRS points to a shapefile
c:\Users\asaf_rs\miniconda3\envs\goesenv\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Unique_date' to 'Unique_dat'
  ogr_write(
c:\Users\asaf_rs\miniconda3\envs\goesenv\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Unique_time' to 'Unique_tim'
  ogr_write(
c:\Users\asaf_rs\miniconda3\envs\goesenv\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'fire_pixel_confidence' to 'fire_pixel'
  ogr_write(
c:\Users\asaf_rs\miniconda3\envs\goesenv\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'fire_pixel_confidence_str' to 'fire_pix_1'
  ogr_wri

In [81]:
fire_point_with_dup = []
for i in range(len(VIIRS_files)):
    VIIRS_gdf = generate_VIIRS_points(VIIRS_files[i])
    fire_point_with_dup.append(VIIRS_gdf)
    
fire_point_with_dup_concat = pd.concat(fire_point_with_dup) ## concatenate the points
fire_point_with_dup_concat = fire_point_with_dup_concat[fire_point_with_dup_concat["fire_pixel_confidence"] != 7]
print(f"the number of fire pixels with duplicates: {len(fire_point_with_dup_concat)} and after removing duplicates: {len(VIIRS_points_concat)} this is a difference of {len(fire_point_with_dup_concat) - len(VIIRS_points_concat)} fire pixels") ## print the total number of fire pixels

the number of fire pixels with duplicates: 247383 and after removing duplicates: 227323 this is a difference of 20060 fire pixels
